** Скрипт, неприменяемый к тестовым данным.**

В определенный момент были догружена часть отсутствующих данных по соц-дему - возраст для всех и соцдцем для некоторых. Этот скрипт соядинеят исходный трейн и догрузку, затем сохраняет все в файл _'dataset.csv'_. 

In [1]:
import pandas as pd
import numpy as np

Исходный датасет:

In [22]:
file_path = './train_data/dataset.csv'
df = pd.read_csv(file_path, sep='>', na_values='<null>', 
                 parse_dates=['START_DATE', 'LAST_CARD_STATUS_DT', 'LAST_CONTR_STATUS_DT'])
print('Dataset size: ', df.shape)

Dataset size:  (191763, 214)


In [ ]:
#df[df['CONTROL_GROUP_FLG']=='N'].count()

In [3]:
df['label'].value_counts()

0    164154
1     27609
Name: label, dtype: int64

In [4]:
df['SCENARIO_NAME_UNI'].value_counts()

POS_3     27250
Sc_4      23276
POS_1     22874
POS_2     20670
UVEL_4    19130
ATM_3     16394
UVEL_1    12577
Sc_1       9502
UVEL_5     8404
Sc_6       7651
ATM_2      7648
ATM_1      5187
UVEL_3     4872
UVEL_6     2972
UVEL_2     2396
Sc_5        642
Sc_2        296
Sc_3         22
Name: SCENARIO_NAME_UNI, dtype: int64

### Догрузка:

#### Возраст

In [5]:
file_path = './train_data/birth.csv'
age = pd.read_csv(file_path, parse_dates=['START_DATE', 'BIRTH_DATE'], dayfirst=True)
age.head()

,CLIENT_ID,CONTRACT_REF,START_DATE,SCENARIO_NAME,BIRTH_DATE
0,105398814,281-P-1027630,2017-03-05,GRAN_UVEL_1_TRUE_BIN,1958-01-14
1,105398814,281-P-1027630,2016-11-23,UVEL_POS_1,1958-01-14
2,105395480,281-P-8372534,2016-11-23,UVEL_POS_1,1964-01-17
3,103277819,271-P-28406103,2017-02-03,UHOD_0102_ATM_3,1977-03-12
4,103274498,271-P-76850624,2017-05-03,KAE_GRAN_UVEL_4_TRUE,1960-12-25


In [6]:
a = age[['CLIENT_ID', 'START_DATE']]
a.head()
#b = a.groupby(['CLIENT_ID']).sum()
#b
# example of duplicate: clientId = 105398814

,CLIENT_ID,START_DATE
0,105398814,2017-03-05
1,105398814,2016-11-23
2,105395480,2016-11-23
3,103277819,2017-02-03
4,103274498,2017-05-03


Соединими данные

In [ ]:
age.shape

MAy be merge by pair = ( CLIENT_ID, START_DATE) ?? !

In [23]:
df = df.merge(
    age[['CLIENT_ID', 'BIRTH_DATE']].drop_duplicates(), 
    how='left', on=['CLIENT_ID'])

# del age
print('Dataset size: ', df.shape)

Dataset size:  (191763, 215)


#### Потерянный соцдем

In [24]:
file_path = './train_data/2017-07-17 miss_soc_dem_result.xlsx'
soc_dem = pd.read_excel(file_path, na_values='<null>')
soc_dem.head()

,client_id_w4,contract_ref,clientid_iq,rep_clid,REP_SEX,FL_GOLD,FL_ZP,FL_PENS,FL_STUD,FL_SOC,...,CL_AMT_ZP,CL_CCY_DOP,CL_AMT_DOP,FL_DIR_UCH,FL_4P,FL_DOC_PROFIT,REP_POPULATION,FL_PENS_DOC,FL_FRNG_PASSP,FL_FRGN_TRIP
0,102237196,273-P-46164631,60437726,60437726,F,N,N,N,N,N,...,0.0,RUR,6000.0,N,N,N,0,N,N,N
1,102312458,271-P-54530230,60624148,60624148,F,N,N,N,N,N,...,10000.0,RUR,0.0,N,N,N,356400,N,N,N
2,104021631,271-P-58512622,60630217,60630217,M,N,N,N,N,N,...,0.0,RUR,0.0,N,N,N,416200,N,N,N
3,102527652,271-P-54443129,1004475980,1004475980,M,Y,N,N,N,N,...,20000.0,NaN,0.0,N,N,N,0,N,N,N
4,103355253,271-P-39321606,1001967290,1001967290,M,Y,N,N,N,N,...,NaN,NaN,NaN,N,N,N,329900,N,N,N


In [25]:
print(len(df.columns))
print(len(df.columns.difference(soc_dem.columns)))

215
185


In [26]:
df['REP_SEX'].value_counts()

F    107522
M     62132
Name: REP_SEX, dtype: int64

In [27]:
df_help = df[df.columns.difference(soc_dem.columns)]\
    .merge(soc_dem.drop(['contract_ref', 'clientid_iq', 'rep_clid', 'QTY_CHLD_16'], axis=1), 
    how='inner', 
    left_on=['CLIENT_ID'], 
    right_on=['client_id_w4']).drop(['client_id_w4'], axis=1)

In [ ]:
print('Dataset size: ', df.shape)
# IT is always True -??
len(df[['CONTRACT_REF', 'START_DATE']]) == df.shape[0]

In [ ]:
set(df.columns) == set(df_help.columns)

In [ ]:
# Concatting by names , not by index
a = pd.DataFrame({'x1':[1,2],
                  'x2':[4,5],
                  'x3':[7,8]})

b = pd.DataFrame({'x2':[10,30],
                  'x1':[-1,-2],
                  'x4':[100,300]})

c = pd.concat([a,b])
c

In [ ]:
#df1 = df.drop_duplicates()
#df1.shape

In [ ]:
#df.groupby(['CONTRACT_REF', 'START_DATE']).count()

Some rows have identical id pair = (contr_ref, start_date), but dofferent features!
How to choose what rows to omit?

In [28]:
df = pd.concat([df, df_help])
print('Dataset size: ', df.shape)

df = df.drop_duplicates(['CONTRACT_REF', 'START_DATE'], keep='last')
print('Dataset size: ', df.shape)

Dataset size:  (198182, 215)
Dataset size:  (191762, 215)


In [ ]:
from datetime import date

def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

age_stat_table = df[df['CONTROL_GROUP_FLG']=='N'][ ['CLIENT_ID', 'START_DATE','CONTRACT_REF', 'BIRTH_DATE','label']]
#age_stat_table.label.value_counts(normalize=True)

age_stat_table['AGE'] = age_stat_table['BIRTH_DATE'].apply(calculate_age)
age_stat_table.drop_duplicates(['CLIENT_ID'], inplace=True)
age_stat_table.drop(['BIRTH_DATE']).to_csv('./train_data/age_stats.csv', 
                                           index=False, sep='>', na_rep='<null>', 
                                           encoding='utf-8')

Удалим леваков (Клиентов со сценарием UHOD)

In [ ]:
df[df['CONTROL_GROUP_FLG']=='N'].shape

In [ ]:
left = pd.read_excel('./train_data/left.xlsx')
left.head(2)

In [ ]:
#df['SCENARIO_NAME_UNI'].value_counts()

In [ ]:
df = df[~df['CONTRACT_REF'].isin(left['CONTRACT_REF'])]

In [ ]:
df.shape

In [ ]:
#df['SCENARIO_NAME_UNI'].value_counts()

#### Новая целевая переменная

In [ ]:
target = pd.read_csv('./train_data/target.csv', parse_dates=['START_DATE'], dayfirst=True)
target.head(5)

In [ ]:
target[target['CONTROL_GROUP_FLG']=='N']['TARGET'].value_counts()

In [ ]:
target.shape

In [ ]:
target.head()

In [ ]:
df = target.merge(df.drop('label', axis=1), how='left', 
    on=['CONTRACT_REF', 'START_DATE', 'CONTROL_GROUP_FLG', 'CLIENT_ID'])
print('Dataset size: ', df.shape)
print('Test group', df[df['CONTROL_GROUP_FLG'] == 'N'].shape[0])
print('Control group', df[df['CONTROL_GROUP_FLG'] == 'Y'].shape[0])

# ПРОВЕРКА НА НОРМАЛЬНОСТЬ ДЖОЙНА

What is it?

In [ ]:
df[pd.isnull(df['BIRTH_DATE'])]['START_DATE'].value_counts(dropna=False)

In [ ]:
df.dropna(subset=['BIRTH_DATE']).to_csv('./train_data/train_dataset.csv', index=False, sep='>', na_rep='<null>', encoding='utf-8')